# Исследование надежности заемщиков, анализ статистики о платежеспособности клиентов банка.

**Описание задачи и имеющихся данных**

Заказчик — кредитный отдел банка. 

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

In [62]:
import pandas as pd
df=pd.read_csv('/datasets/data.csv')
display(df.head())

df.info()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

In [63]:
display(df.describe())
print('Процент пропущенных значений в колонках days_employed и total_income:',(21525-19351)/21525*100)

df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Процент пропущенных значений в колонках days_employed и total_income: 10.099883855981417


Пропущены числовые значения в столбцах с количеством рабочих дней и итоговым доходом в одних и тех же строках. Пропуск можно объяснить тем, что данные могли быть не предосавлены на этапе оформления кредита.
Замену пропущенных числовых данных в столбце с итоговым доходом лучше выполнить на медианное значение с учетом категории дохода из столбца income_type.
Среднее значение даст некорректный результат, поскольку в слобце есть очень высокие и очень низкие значения, которые могут исказить результат.


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [64]:
print(df['children'].value_counts())


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В столбце children имеются аномальные значения: "20" и "-1". Оба эти значения могли появиться из-за неправильного ввода данных (20 - это 2 с лишним нулем, -1 - это 1)
Исправим эти значения методом .replace() на 2 и 1 соответсвенно.

In [65]:
df['children']=df['children'].replace(20,2)
df['children']=df['children'].replace(-1,1)
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [66]:
display(df['days_employed'].describe())


count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

В столбце days_employed помимо пропусков есть отрицательные и аномально высокие значения (1100 лет).
Отрицательные значения заменим методом .abs()
Сделаем срез по столбцу days_employed со значениями менее 60 лет и выведем общую информацию о нем методом .describe() и заменим значения более 60 лет и пропуски медианой этого среза

In [67]:
df['days_employed']=df['days_employed'].abs()
print(df.loc[df['days_employed']<(60*365),'days_employed'].describe())
df['days_employed']=df['days_employed'].where(
    df['days_employed']<(60*365),df.loc[df['days_employed']<(60*365),'days_employed'].median()
)
df['days_employed'].describe()

count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64


count    21525.000000
mean      2164.281083
std       2006.061675
min         24.141633
25%       1025.608174
50%       1630.019381
75%       2518.168900
max      18388.949901
Name: days_employed, dtype: float64

В столбце dob_years 101 нулевое значение. Поскольку определить возраст заемщика по имеющимся данным не получится, удаляем строки с нулевым значением dob_years

In [68]:
print(df['dob_years'].sort_values().value_counts())
df=df.loc[df['dob_years']!=0]



35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


В столбцах education_id, family_status_id и debt аномальных значений не выявлено

In [69]:
print(df['education_id'].value_counts())
print()
print(df['family_status_id'].value_counts())
print()
print (df['debt'].value_counts())


1    15169
0     5225
2      742
3      282
4        6
Name: education_id, dtype: int64

0    12331
1     4156
4     2797
3     1185
2      955
Name: family_status_id, dtype: int64

0    19691
1     1733
Name: debt, dtype: int64


### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype()

In [70]:
df['total_income']=df['total_income'].astype('int64')
df['days_employed']=df['days_employed'].astype('int64')
print(
    df['total_income'].dtypes,
    df['days_employed'].dtypes
)

int64 int64


### Шаг 2.4. Удаление дубликатов

Проверим наличие строк-дупликатов методом .duplicated() и удалим найденные строки методом .drop_duplicates()

In [71]:
print(df.duplicated().sum())
df=df.drop_duplicates()
df.duplicated().sum()

54


0

Проверим наличие неявных дупликатов методом .value_counts()

In [72]:
print(df['education'].value_counts())
print()
print(df['family_status'].value_counts())
print()
print(df['gender'].value_counts())
print()
print(df['income_type'].value_counts())
print()
print(df['purpose'].value_counts())

среднее                13646
высшее                  4678
СРЕДНЕЕ                  770
Среднее                  708
неоконченное высшее      666
ВЫСШЕЕ                   272
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

женат / замужем          12295
гражданский брак          4142
Не женат / не замужем     2794
в разводе                 1185
вдовец / вдова             954
Name: family_status, dtype: int64

F      14117
M       7252
XNA        1
Name: gender, dtype: int64

сотрудник          11036
компаньон           5060
пенсионер           3817
госслужащий         1451
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   78

Удалим строку со значением 'XNA' в столбце gender

In [73]:
df=df.loc[df['gender']!='XNA']
df['gender'].value_counts()

F    14117
M     7252
Name: gender, dtype: int64

Избавимся от дубликатов в колонках education и family_status, приведя их значения к нижнему регистру методом .str.lower()

In [74]:
df['education']=df['education'].str.lower()
df['family_status']=df['family_status'].str.lower()
print(df['education'].value_counts())
print()
print(df['family_status'].value_counts())

среднее                15124
высшее                  5216
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12295
гражданский брак          4141
не женат / не замужем     2794
в разводе                 1185
вдовец / вдова             954
Name: family_status, dtype: int64


Причиной возникновения неявных дубликатов в виде данных, записанных в разном регистре может быть разница в том, как разные операторы набирают данные при внесении в базу или разница в настройках форм записи конкретного приложения.

Повторим проверку на явные дубликаты.

In [75]:
print(df.duplicated().sum())
df=df.drop_duplicates()
df.duplicated().sum()

17


0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Cоздаем отдельные "словари" для идентификаторов значений колонок education и family_status.
Удаляем соответсвующие столбцы из исходного датафрейма методом .drop()

In [76]:
education_dict=df[['education','education_id']].drop_duplicates().reset_index(drop=True)
family_status_dict=df[['family_status','family_status_id']].drop_duplicates().reset_index(drop=True)
display(education_dict,family_status_dict)

df=df.drop(['education','family_status'],1)
df.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,1630,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

Пишем функцию total_income_category(), которая возвращает категории дохода в зависимости от его размера.
Создаем в датафрейме новый столбец. С помощью метода apply() заполняем этот столбец значениями, которые вернула функция total_income_category со значением параметра из столбца total_income.


In [77]:
def total_income_category(income):
    if income <=30000:
        return 'E'
    if income <=50000:
        return 'D'
    if income <=200000:
        return 'C'
    if income <=1000000:
        return 'B'
    return 'A'
df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,1630,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Для того чтобы добавить категории для цели кредита - пишем функцию purpose_category_func(row), которая возвращает значение категории в зависимости от значения в столбце purpose. 
Создаем столбец purpose_category и применяем к нему функцию методом .apply()

In [78]:
def purpose_category_func(row):
    if 'жиль' in row:
        return 'операции с недвижимостью'
    elif 'недвиж' in row:
        return 'операции с недвижимостью'
    elif 'свад' in row:
        return 'проведение свадьбы'
    elif 'авто' in row:
        return 'операции с автомобилем'
    else:
        return 'получение образования'

df['purpose_category'] = df['purpose'].apply(purpose_category_func)
df.head()
  

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,1630,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [79]:
print(df['debt'].mean())
def pivot(column):
    return df.pivot_table(index=[column],values='debt',aggfunc=['count','sum','mean'])
    
    
pivot('children')


0.08116335706257025


,count,sum,mean
,debt,debt,debt
children,,,
0,14021,1058,0.075458
1,4839,442,0.091341
2,2114,202,0.095553
3,328,27,0.082317
4,41,4,0.097561
5,9,0,0.000000


##### Вывод 1:

Из сводной таблицы по значениям 'debt' у заемщиков с различным количеством детей видно, что заемщики без детей в среднем реже не платят по кредиту.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [80]:
display(family_status_dict)
pivot('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


,count,sum,mean
,debt,debt,debt
family_status_id,,,
0,12290,927,0.075427
1,4129,386,0.093485
2,954,62,0.064990
3,1185,85,0.071730
4,2794,273,0.097709


##### Вывод 2: 

Из сводной таблицы по значениям 'debt' у заемщиков с различным семейным положением прослеживается связь: среднее количество невозвратов примерно на 2 % выше у заемщиков со статусами 'гражданский брак' и 'не женат / не замужем'

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [81]:
pivot('total_income_category')

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5015,355,0.070788
C,15942,1353,0.084870
D,348,21,0.060345
E,22,2,0.090909


##### Вывод 3:

Из сводной таблицы по значениям 'debt' у заемщиков с различным уровнем дохода зависимость не прослеживается.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [82]:
pivot('purpose_category')

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4284,400,0.093371
операции с недвижимостью,10763,779,0.072378
получение образования,3995,370,0.092616
проведение свадьбы,2310,184,0.079654


##### Вывод 4:

Из сводной таблицы по значениям 'debt' у заемщиков с различной целью кредита прослеживается зависимость - примерно на 2 % больше невозвратов у заемщиков, берущих кредиты на получение образования и покупку автомобиля.

## Общий вывод:

Предлагалось определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Есть влияние семейного пложения на факт погашения кредита. В целом среднее количество невозвратов примерно на 2 % выше у заемщиков со статусами 'гражданский брак' и 'не женат / не замужем'.

Также выявлено, что бездетные заемщики в среднем реже не платят по кредиту, чем люди с 1 ребенком и более.

Также выявлены дополнительные факторы, способные повлиять на факт возврата кредита: у людей, берущих кредиты на недвижимость или свадьбу - доля невозвратов в среднем ниже. Также доля невозвратов больше среди людей с низким доходом.

Таким образом портрет "идеального" заемщика был бы следующий: женатый бездетный человек с доходом от 50 000 рублей с целью кредита - операции с недвижимостью или проведение свадьбы.

